# 11 XPCS Presets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/08_workflow_system/11_xpcs_presets.ipynb)

XPCS (X-ray Photon Correlation Spectroscopy) Domain Preset Example.

This example demonstrates how to create a custom preset for XPCS data analysis
using NLSQ's kwargs factory pattern.

XPCS analysis typically involves:
- Fitting correlation functions (g2) to extract relaxation times
- Multi-scale parameters (tau can span nanoseconds to hours)
- High precision requirements for publication-quality results
- Potential for large datasets from modern 2D detectors

Run this example:
    python examples/scripts/08_workflow_system/11_xpcs_presets.py

In [1]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

Not running in Colab - assuming NLSQ is already installed


In [2]:
import jax.numpy as jnp
import numpy as np

from nlsq import fit

In [3]:
def create_xpcs_preset() -> dict:
    """Create a workflow configuration optimized for XPCS correlation function fitting.

    XPCS-specific considerations:
    - Tolerances: 1e-8 provides sufficient precision for correlation functions
    - Multi-start: Enabled to avoid local minima in stretched exponential fits
    - Normalization: Important for parameters spanning many orders of magnitude

    Returns
    -------
    dict
        kwargs for fit() optimized for XPCS analysis.

    Example
    -------
    >>> kwargs = create_xpcs_preset()
    >>> kwargs['gtol']
    1e-08
    >>> kwargs['multistart']
    True
    """
    return {
        "workflow": "standard",
        "multistart": True,
        "n_starts": 15,
        "sampler": "sobol",
        "gtol": 1e-8,
        "ftol": 1e-8,
        "xtol": 1e-8,
    }


def xpcs_g2_model(t, tau, beta, baseline, contrast):
    """XPCS correlation function model (g2 - 1).

    The normalized intensity autocorrelation function:
        g2(t) - 1 = contrast * exp(-2 * (t/tau)^beta)

    Parameters
    ----------
    t : array_like
        Delay times (typically logarithmically spaced)
    tau : float
        Relaxation time (characteristic decay time)
    beta : float
        Stretching exponent (0 < beta <= 1 for subdiffusive, beta > 1 for superdiffusive)
    baseline : float
        Baseline offset (ideally 0, but may drift in practice)
    contrast : float
        Speckle contrast (ideally close to theoretical maximum)

    Returns
    -------
    array
        g2(t) - 1 values at each delay time
    """
    return baseline + contrast * jnp.exp(-2.0 * (t / tau) ** beta)

In [4]:
def main():
    print("=" * 70)
    print("XPCS Domain Preset Example")
    print("=" * 70)
    print()

    # Create the XPCS preset
    kwargs = create_xpcs_preset()

    print("XPCS Preset Configuration:")
    print("-" * 40)
    for key, value in kwargs.items():
        print(f"  {key}: {value}")
    print()

    # Generate synthetic XPCS data
    print("Generating synthetic XPCS data...")
    np.random.seed(42)

    t_data = np.logspace(-6, 2, 100)

    true_params = {
        "tau": 0.01,
        "beta": 0.8,
        "baseline": 0.0,
        "contrast": 0.3,
    }

    y_true = xpcs_g2_model(
        t_data,
        true_params["tau"],
        true_params["beta"],
        true_params["baseline"],
        true_params["contrast"],
    )
    noise = 0.01 * np.random.randn(len(t_data))
    y_data = y_true + noise

    print(f"  Data points: {len(t_data)}")
    print(f"  True tau:    {true_params['tau']:.4f} s")
    print(f"  True beta:   {true_params['beta']:.2f}")
    print()

    p0 = [0.1, 0.9, 0.0, 0.25]
    bounds = (
        [1e-8, 0.1, -0.1, 0.01],
        [1e3, 2.0, 0.1, 1.0],
    )

    print("Fitting with XPCS preset...")
    popt, pcov = fit(
        xpcs_g2_model,
        t_data,
        y_data,
        p0=p0,
        bounds=bounds,
        **kwargs,
    )

    print()
    print("Fit Results:")
    print("-" * 40)
    print(f"  tau:      {popt[0]:.6f} s (true: {true_params['tau']:.6f})")
    print(f"  beta:     {popt[1]:.4f} (true: {true_params['beta']:.4f})")
    print(f"  baseline: {popt[2]:.6f} (true: {true_params['baseline']:.6f})")
    print(f"  contrast: {popt[3]:.4f} (true: {true_params['contrast']:.4f})")

    if pcov is not None:
        perr = np.sqrt(np.diag(pcov))
        print()
        print("Parameter Uncertainties (1-sigma):")
        print(f"  tau:      +/- {perr[0]:.6f} s")
        print(f"  beta:     +/- {perr[1]:.6f}")
        print(f"  baseline: +/- {perr[2]:.6f}")
        print(f"  contrast: +/- {perr[3]:.6f}")

    print()
    print("=" * 70)
    print("Notes on XPCS Preset Customization")
    print("=" * 70)
    print()
    print("The XPCS preset uses the kwargs factory pattern with adjustments for:")
    print()
    print("1. Multi-start optimization (n_starts=15)")
    print("   - Stretched exponential fits often have multiple local minima")
    print("   - Especially important when beta is unknown a priori")
    print()
    print("2. Sobol sampling")
    print("   - Better coverage of the tau/beta parameter space")
    print("   - More uniform exploration than random sampling")
    print()
    print("3. Parameter bounds guidance:")
    print("   - tau: Set bounds based on experimental time window")
    print("   - beta: [0.1, 2.0] covers sub- and super-diffusive regimes")
    print("   - contrast: [0, 1] for normalized correlation functions")
    print()

In [5]:
if __name__ == "__main__":
    main()

XPCS Domain Preset Example

XPCS Preset Configuration:
----------------------------------------
  workflow: standard
  multistart: True
  n_starts: 15
  sampler: sobol
  gtol: 1e-08
  ftol: 1e-08
  xtol: 1e-08

Generating synthetic XPCS data...


INFO:nlsq.multi_start:Generating 15 starting points using sobol n_starts=15 | sampler=sobol | center_on_p0=True


  Data points: 100
  True tau:    0.0100 s
  True beta:   0.80

Fitting with XPCS preset...


INFO:nlsq.multi_start:Evaluating 15 starting points


INFO:nlsq.curve_fit:Starting curve fit n_params=4 | n_data_points=100 | method=trf | solver=auto | batch_size=None | has_bounds=True | dynamic_sizing=False


...

INFO:nlsq.least_squares:Convergence reason=`ftol` termination condition is satisfied. | iterations=19 | final_cost=0.0040 | elapsed=0.344s | final_gradient_norm=6.6226e-08


PERFORMANCE:nlsq.curve_fit:Timer: curve_fit elapsed=0.421854s


INFO:nlsq.curve_fit:Curve fit completed total_time=0.4219 | final_cost=0.0080 | covariance_warning=False


INFO:nlsq.multi_start:Best starting point: loss=0.007976 best_loss=0.0080 | best_params=[0.009860307757420892, 0.7830704280977765, 9.023640697329227e-05, 0.2991428002446006]



Fit Results:
----------------------------------------
  tau:      0.009860 s (true: 0.010000)
  beta:     0.7831 (true: 0.8000)
  baseline: 0.000090 (true: 0.000000)
  contrast: 0.2991 (true: 0.3000)

Parameter Uncertainties (1-sigma):
  tau:      +/- 0.000500 s
  beta:     +/- 0.028525
  baseline: +/- 0.001353
  contrast: +/- 0.002548

Notes on XPCS Preset Customization

The XPCS preset uses the kwargs factory pattern with adjustments for:

1. Multi-start optimization (n_starts=15)
   - Stretched exponential fits often have multiple local minima
   - Especially important when beta is unknown a priori

2. Sobol sampling
   - Better coverage of the tau/beta parameter space
   - More uniform exploration than random sampling

3. Parameter bounds guidance:
   - tau: Set bounds based on experimental time window
   - beta: [0.1, 2.0] covers sub- and super-diffusive regimes
   - contrast: [0, 1] for normalized correlation functions

